In [16]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [255]:
import psycopg2
import pandas as pd
from datetime import timedelta, date
from dateutil.parser import parse
from IPython.display import display, HTML
import careLifeSecrets

def get_week_dates(base_date, start_day, end_day=None):
    monday = base_date - timedelta(days=base_date.isoweekday() - 1)
    week_dates = [monday + timedelta(days=i) for i in range(7)]
    return week_dates[start_day - 1:end_day or start_day][0]

weekStartDate = get_week_dates(date.today(), 1, 8)

query = "SELECT sensor_info FROM public.audit_emfit_info where sensor_id = 'insert_sensor_id' and info_type = '2' and action->>'action' = 'UPDATE' and sensor_time > 'insert_week_start_date' ORDER BY sensor_time DESC"

connection = psycopg2.connect(database = careLifeSecrets.DATABASE , user= careLifeSecrets.USER , password = careLifeSecrets.PG_PASSWORD[:-1] , host = careLifeSecrets.HOST, port = careLifeSecrets.PORT)
print("Opened Care.Life DB successfully")
trimmedSleepData, tempDict = [],{}

sensor_id = input("Enter sensor_id : ")

query = query.replace("insert_sensor_id", sensor_id)
query = query.replace("insert_week_start_date", str(weekStartDate))

cursor1 = connection.cursor()
cursor1.execute(query)
rows = cursor1.fetchall()

query2 = "SELECT sensor_info FROM public.emfit_info where sensor_id = 'insert_sensor_id' and info_type = '2' ORDER BY sensor_time DESC;"
query2 = query2.replace("insert_sensor_id", sensor_id)
cursor2 = connection.cursor()
cursor2.execute(query2)
LatestSleep = cursor2.fetchall()

rows.insert(0,LatestSleep[0])

query3 = "select time_zone from sensor_mapping where sensor_id =" + " sensor_id " + "limit 1;"
cursor3 = connection.cursor()
cursor3.execute(query3)
time_zone = cursor3.fetchall()[0][0]
print(time_zone)

Opened Care.Life DB successfully
Enter sensor_id : 003ED5
America/Denver


In [256]:
from pytz import timezone
import datetime

trimmedSleepData, tempDict = [],{}
dataPointsToOmmit = ['hrv_data','sleep_score','id','device','bedexit_data','from_gmt_offset','avg_act','calc_data','sleep_data','tossnturn_data','hrv_rmssd_data','fm_data','hrv_hf','hrv_lf','fm_count','hrv_score','duration_in_rem','duration_in_light','duration_in_deep','duration_sleep_onset']      #Edit this list to choose what sensor data to ignore

for row in rows:
    for k,v in row[0].items():
        if k not in dataPointsToOmmit:
            if k == 'to' or k == 'from':
                utcmoment = datetime.datetime.utcfromtimestamp(v).replace(tzinfo=pytz.utc)
                tempDict[k] = utcmoment.astimezone(pytz.timezone(time_zone)).strftime('%Y-%m-%d %H:%M:%S')
                if parse(tempDict[k]) <= datetime.datetime.now() - datetime.timedelta(days=6):
                    tempDict = {}
                    breakParsing = True
                    break
            elif k == 'duration_awake' or k == 'bedexit_duration' or k == 'duration_in_bed' or k == 'duration_in_sleep' or k == 'duration':
                tempDict[k] = v
            elif k == 'avg_hr':
                tempDict['average heart rate'] = v
            elif k == 'avg_rr':
                tempDict['average respiration rate'] = v
            elif k == 'min_hr':
                tempDict['minimum heart rate'] = v
            elif k == 'min_rr':
                tempDict['minimum respiration rate'] = v
            elif k == 'max_hr':
                tempDict['maximum heart rate'] = v
            elif k == 'bedexit_count':
                tempDict['number of bed exits'] = v
            elif k == 'max_rr':
                tempDict['maximum respiration rate'] = v
            elif k == 'bedexit_count':
                tempDict['number of bed exits'] = v
            else:
                tempDict[k] = v
    if tempDict not in trimmedSleepData:
        trimmedSleepData.append(tempDict)
    tempDict = {}

In [258]:
display(LatestSleep[0])

({'id': '64218981ef41ac2355a63a91',
  'to': 1679919398,
  'from': 1679895253,
  'avg_hr': 68.8,
  'avg_rr': 14.2,
  'device': '003ED5',
  'hrv_hf': 53,
  'hrv_lf': 47,
  'max_hr': 78,
  'max_rr': 19,
  'min_hr': 53,
  'min_rr': 6,
  'avg_act': 21.5,
  'fm_data': '',
  'duration': 24145,
  'fm_count': 0,
  'hrv_score': 27,
  'awakenings': 2,
  'sleep_score': 65,
  'bedexit_count': 1,
  'duration_awake': 2490,
  'duration_in_bed': 24016,
  'duration_in_rem': 5970,
  'from_gmt_offset': 180,
  'tossnturn_count': 11,
  'bedexit_duration': 129,
  'duration_in_deep': 3060,
  'duration_in_light': 12630,
  'duration_in_sleep': 21660,
  'hrv_rmssd_evening': 14.5,
  'hrv_rmssd_morning': 19,
  'duration_sleep_onset': 750},)

# overlap Fix

In [259]:
fromList = []
correctedSleepData = []
for i in trimmedSleepData:
    if 'from' in i.keys():
        if i['from'] in fromList:
            continue
        else:
            correctedSleepData.append(i)
            fromList.append(i['from'])

In [260]:
display(trimmedSleepData)

[{'to': '2023-03-27 06:16:38',
  'from': '2023-03-26 23:34:13',
  'average heart rate': 68.8,
  'average respiration rate': 14.2,
  'maximum heart rate': 78,
  'maximum respiration rate': 19,
  'minimum heart rate': 53,
  'minimum respiration rate': 6,
  'duration': 24145,
  'awakenings': 2,
  'number of bed exits': 1,
  'duration_awake': 2490,
  'duration_in_bed': 24016,
  'duration_in_rem': 5970,
  'tossnturn_count': 11,
  'bedexit_duration': 129,
  'duration_in_deep': 3060,
  'duration_in_light': 12630,
  'duration_in_sleep': 21660,
  'hrv_rmssd_evening': 14.5,
  'hrv_rmssd_morning': 19,
  'duration_sleep_onset': 750}]

In [270]:
from statistics import mean
import datetime
from math import ceil

AverageSleepDuration = []
AverageTimeInBed = []
AverageBedExitCount = []
AverageHeartRate = []
AverageRespirationRate = []
AverageBestExitDuration = []
AnamolyList = []
SleepDurationByDay = {}
inBedDuratonByDay = {}

RiskList = {}
MinAvgRespRate = 9
MaxAvgRespRate = 25
MinAvgHeartpRate = 50
MaxAvgHeartRate = 100
MinRespRate = 8
MaxRespRate = 30
MinHeartRate = 50
MaxHeartRate = 100
MaxBedExitDuration = 40
MaxBedExitCount = 0
MaxBedExitsOn = ''

for row in trimmedSleepData:
    if row and 'duration_in_sleep' in row.keys() and 'duration_in_bed' in row.keys() :
        print(row['to'])
        print(row['from'])

        if row['to'][:10] in SleepDurationByDay.keys() and 'duration_in_sleep' in row.keys():
            SleepDurationByDay[row['to'][:10]] = SleepDurationByDay[row['to'][:10]] + row['duration_in_sleep']
        else:
            SleepDurationByDay[row['to'][:10]] = row['duration_in_sleep']
       
            
        if row['to'][:10] in inBedDuratonByDay.keys() and 'duration_in_bed' in row.keys():
            inBedDuratonByDay[row['to'][:10]] = inBedDuratonByDay[row['to'][:10]] + row['duration_in_bed']
        else:
            inBedDuratonByDay[row['to'][:10]] = row['duration_in_bed']
            
        if 'number of bed exits' in row.keys() and 'average heart rate' in row.keys() and 'average respiration rate' in row.keys() and 'bedexit_duration' in row.keys():
            if row['number of bed exits'] > MaxBedExitCount:
                MaxBedExitCount = row['number of bed exits']
                MaxBedExitsOn = str(row['from'][:10])
            AverageBedExitCount.append(row['number of bed exits'])
            AverageHeartRate.append(row['average heart rate'])
            AverageRespirationRate.append(row['average respiration rate'])
            AverageBestExitDuration.append(row['bedexit_duration'])
    
        if 'minimum respiration rate' in row.keys():
            if row['average respiration rate'] < MinAvgRespRate:
                MinAvgRespRate = row['average respiration rate']
                RiskList['low average respiratory rate'] = "low average Respiratory Rate of " + str(MinAvgRespRate) + " on " + str(row['to'])
                
            if row['average heart rate'] < MinAvgHeartpRate:
                MinAvgHeartpRate = row['average heart rate']
                RiskList['high average heart rate'] = "low average heart Rate of " + str(MinAvgHeartRate) + " on " + str(row['to'])
                
            if row['average respiration rate'] > MaxAvgRespRate:
                MaxAvgRespRate = row['average respiration rate']
                RiskList['low average respiratory rate'] = "high average Respiratory Rate of " + str(MaxAvgRespRate) + " on " + str(row['to'])
                
            if row['average heart rate'] > MaxAvgHeartRate:
                MaxAvgHeartpRate = row['average heart rate']
                RiskList['high average heart rate'] = "high average heart Rate of " + str(MaxAvgHeartRate) + " on " + str(row['to'])
        
            if row['minimum respiration rate'] < MinRespRate:
                MinRespRate = row['minimum respiration rate']
                RiskList['low respiratory rate'] = "low Respiratory Rate of " + str(MinRespRate) + " on " + str(row['to'])
        
            if row['maximum respiration rate'] > MaxRespRate:
                MaxRespRate = row['maximum respiration rate']
                RiskList['high respiratory rate'] = "high Respiratory Rate of " + str(MaxRespRate) + " on " + str(row['to'][:10])
        
            if row['minimum heart rate'] < MinHeartRate:
                MinHeartRate = row['minimum heart rate']
                RiskList['low heart rate'] = "very low Heart Rate of " + str(MinHeartRate) + " on " + str(row['to'][:10])
        
            if row['maximum heart rate'] > MaxHeartRate:
                MaxHeartRate = row['maximum heart rate']
                RiskList['high heart rate'] = "high Heart Rate of " + str(MaxHeartRate) + " on " + str(row['to'][:10])
        
            if row['bedexit_duration']/60 > 30:
                MaxBedExitDuration = "{0:.2f}".format(row['bedexit_duration']/60)
                RiskList['Unusual bed exit duration'] = "Unusually long bed exit duration of " + str(MaxBedExitDuration) + " minutes on " + str(row['to'][:10])
        
            if row['duration_in_sleep']/3600 < 7:
                RiskList['low sleep duration'] = "Sleep duration was lower than usual (" + str("{0:.1f}".format(row['duration_in_sleep']/3600)) + " Hours ) on " + str(row['to'])
                
            if row['duration_in_sleep']/3600 > 12:
                RiskList['high sleep duration'] = "Sleep duration was higher than usual (" + str("{0:.1f}".format(row['duration_in_sleep']/3600)) + " Hours ) on " + str(row['to'])
                
for i in SleepDurationByDay.keys():
    print(str(i) + " : " +  str(SleepDurationByDay[i]/3600))
    
print('average sleep time : ' + str("{0:.2f}".format(mean(SleepDurationByDay.values())/3600)) + 'Hours')
print('average duration in bed : ' + str("{0:.2f}".format(mean(inBedDuratonByDay.values())/3600)) + 'Hours')
print('average heart rate : ' +  str(ceil(mean(AverageHeartRate))))
print('average respiration rate : ' +  str(ceil(mean(AverageRespirationRate))))
print('average bed exit count : ' + str(ceil(mean(AverageBedExitCount))))
print('Max bed Exits  : ' + str(MaxBedExitCount) + ' on ' + MaxBedExitsOn)
print('Abnormality List :' + str(RiskList))

postPrompt = 'average sleep time : ' + str("{0:.2f}".format(mean(SleepDurationByDay.values())/3600)) + 'Hours' + "\n" + 'average duration in bed : ' + str("{0:.2f}".format(mean(inBedDuratonByDay.values())/3600)) + 'Hours' + "\n" + 'average heart rate : ' +  str(ceil(mean(AverageHeartRate))) + "\n" + 'average respiration rate : ' +  str(ceil(mean(AverageRespirationRate))) + "\n" + 'average bed exit count : ' + str(ceil(mean(AverageBedExitCount))) + "\n" +  'Max bed Exits  : ' + str(MaxBedExitCount) + ' on ' + MaxBedExitsOn + "\n" + 'Abnormality List :' + str(RiskList) 

2023-03-27 06:16:38
2023-03-26 23:34:13
2023-03-27 : 6.016666666666667
average sleep time : 6.02Hours
average duration in bed : 6.67Hours
average heart rate : 69
average respiration rate : 15
average bed exit count : 1
Max bed Exits  : 1 on 2023-03-26
Abnormality List :{'low respiratory rate': 'low Respiratory Rate of 6 on 2023-03-27 06:16:38', 'low sleep duration': 'Sleep duration was lower than usual (6.0 Hours ) on 2023-03-27 06:16:38'}


In [262]:
correctedSleepData[0]

{'to': '2023-03-27 06:16:38',
 'from': '2023-03-26 23:34:13',
 'average heart rate': 68.8,
 'average respiration rate': 14.2,
 'maximum heart rate': 78,
 'maximum respiration rate': 19,
 'minimum heart rate': 53,
 'minimum respiration rate': 6,
 'duration': 24145,
 'awakenings': 2,
 'number of bed exits': 1,
 'duration_awake': 2490,
 'duration_in_bed': 24016,
 'duration_in_rem': 5970,
 'tossnturn_count': 11,
 'bedexit_duration': 129,
 'duration_in_deep': 3060,
 'duration_in_light': 12630,
 'duration_in_sleep': 21660,
 'hrv_rmssd_evening': 14.5,
 'hrv_rmssd_morning': 19,
 'duration_sleep_onset': 750}

In [263]:
from dateutil.parser import parse
import datetime
dt = parse(trimmedSleepData[0]['to'])
date = datetime.datetime.now() - datetime.timedelta(days=7)
print(date)
print(dt > date)

2023-03-20 12:49:59.304989
True


In [264]:
display(trimmedSleepData)

[{'to': '2023-03-27 06:16:38',
  'from': '2023-03-26 23:34:13',
  'average heart rate': 68.8,
  'average respiration rate': 14.2,
  'maximum heart rate': 78,
  'maximum respiration rate': 19,
  'minimum heart rate': 53,
  'minimum respiration rate': 6,
  'duration': 24145,
  'awakenings': 2,
  'number of bed exits': 1,
  'duration_awake': 2490,
  'duration_in_bed': 24016,
  'duration_in_rem': 5970,
  'tossnturn_count': 11,
  'bedexit_duration': 129,
  'duration_in_deep': 3060,
  'duration_in_light': 12630,
  'duration_in_sleep': 21660,
  'hrv_rmssd_evening': 14.5,
  'hrv_rmssd_morning': 19,
  'duration_sleep_onset': 750}]

Preparing Data for Fine tuning

In [215]:
!pip install --upgrade openai

In [220]:
import pandas as pd

df = pd.read_csv('Book.csv')
df.to_json("SleepDataFineTuning.jsonl", orient='records', lines=True)

In [217]:
import os
import openai
openai.api_key = os.getenv(environmentVariables.OPENAI_API_KEY)
!openai tools fine_tunes.prepare_data -f SleepDataFineTuning.jsonl -q
#!openai api fine_tunes.create -t <TRAIN_FILE_ID_OR_PATH> -m davinci

Analyzing...

- Your file contains 10 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix `'} `
- All prompts start with prefix `average sleep time : `. Fine-tuning doesn't require the instruction specifying the task, or a few-shot example scenario. Most of the time you should only add the input data into the prompt, and the desired output into the completion
- All completions start with prefix `The resident slept an average of `. Most of the time you should only add the output data into the completion, without any prefix
- Your data does not contain a common ending at the end of your completions. Having a common ending string appended to the end of the completion makes it clearer to the fine-tuned model where the completion should end. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for m

In [226]:
!openai api fine_tunes.create -t "SleepDataFineTuning.jsonl" -m davinci 


Found potentially duplicated files with name 'SleepDataFineTuning.jsonl', purpose 'fine-tune' and size 14337 bytes
file-xghVXarEzWT7YrCFmFsHJHrI
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: ^C



In [228]:
!openai api fine_tunes.follow -i ft-Qp3XcUtQKv5Vjn7OqO7nhvjt

[2023-03-23 14:34:47] Created fine-tune: ft-Qp3XcUtQKv5Vjn7OqO7nhvjt
[2023-03-23 14:38:18] Fine-tune costs $0.44
[2023-03-23 14:38:19] Fine-tune enqueued. Queue number: 3
[2023-03-23 14:38:36] Fine-tune is in the queue. Queue number: 2
[2023-03-23 14:39:30] Fine-tune is in the queue. Queue number: 1
[2023-03-23 14:40:08] Fine-tune is in the queue. Queue number: 0
[2023-03-23 14:41:15] Fine-tune started
[2023-03-23 14:42:52] Completed epoch 1/4
[2023-03-23 14:42:59] Completed epoch 2/4
[2023-03-23 14:43:05] Completed epoch 3/4
[2023-03-23 14:43:10] Completed epoch 4/4
[2023-03-23 14:43:58] Uploaded model: davinci:ft-care-life-inc-2023-03-23-20-43-58
[2023-03-23 14:43:59] Uploaded result file: file-TNJwclREiq7Lu2qTSyUihx6z
[2023-03-23 14:43:59] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m davinci:ft-care-life-inc-2023-03-23-20-43-58 -p <YOUR_PROMPT>


In [265]:

import openai
import os
os.environ["OPENAI_API_KEY"] = careLifeSecrets.OPENAI_API_KEY
!openai api completions.create -m davinci:ft-care-life-inc-2023-02-27-23-54-36 -p "Generate a human-readable sleep summary report from the following sleep data from an elderly adult average sleep time : 11.13Hours average duration in bed : 11.87Hours average heart rate : 61.78 average respiration rate : 14.76 average bed exit count : 4.44 Abnormality List :{'low Respiratory Rate': 'low Respiratory Rate of 7 on 2023-02-22 21:59:54', 'high Respiratory Rate': '', 'low Heart Rate': 'very low Heart Rate of 39 on 2023-02-22', 'high heart Rate': 'high Heart Rate of 122 on 2023-02-24', 'Unusual bed exit duration': 'Unusually long bed exit duration of 51.62 minutes on 2023-02-27', 'duration_in_sleep': 'Sleep duration was lower than usual on 2023-02-22 21:59:54'}"


Generate a human-readable sleep summary report from the following sleep data from an elderly adult average sleep time : 11.13Hours average duration in bed : 11.87Hours average heart rate : 61.78 average respiration rate : 14.76 average bed exit count : 4.44 Abnormality List :{'low Respiratory Rate': 'low Respiratory Rate of 7 on 2023-02-22 21:59:54', 'high Respiratory Rate': '', 'low Heart Rate': 'very low Heart Rate of 39 on 2023-02-22', 'high heart Rate': 'high Heart Rate of 122 on 2023-02-24', 'Unusual bed exit duration': 'Unusually long bed exit duration of 51.62 minutes on 2023-02-27', 'duration_in_sleep': 'Sleep duration was lower than usual on 2023-02-22 21:59:54'}} Sleep Quality Summary : ( Average Sleep Quality : Normal ) Overall Sleep pattern is

In [279]:
ft_model = 'davinci:ft-care-life-inc-2023-03-23-20-43-58'
openai.api_key = careLifeSecrets.OPENAI_API_KEY

prePrompt = "average sleep time : 14.7 Hours average duration in bed : 15.6Hours average heart rate : 53.9 average respiration rate : 20.8 average bed exit count : 1.5 Max bed Exits : 3 on 2023-03-15 Abnormality List :{'low Respiratory Rate': 'low Respiratory Rate of 5 on 2023-03-21 13:03:39', 'high Respiratory Rate': '', 'low Heart Rate': 'very low Heart Rate of 38 on 2023-03-17', 'high heart Rate': '', 'Unusual bed exit duration': '', 'duration_in_sleep': 'Sleep duration was lower than usual(5.11 Hours ) on 2023-03-18 06:17:48'} \n The resident slept an average of 14.7 hours a day over the past week which is above the recommended range of 7-12 hours. Additionally, they spent an average of 15.6 hours per day in their bed. On Saturday, March 18th the sleep duration was 5 hours which is lower than the recommended range. The resident exited the bed 1.5 times per night with the most bed exits occurring on Wednesday, March 15th with a total of 3. The heart rate throughout the week averaged 53.9 beats per minute which falls within the normal range of 50-100 beats per minute for an elderly adult. On Friday, March 17th the resident’s heart rate was at its lowest reaching 38 beats per minute. The respiratory rate throughout the week averaged at 20.8 breaths per minute, which is within the normal range of 8-30. On Tuesday, March 21st the resident’s respiratory rate was at its lowest reaching 5 breaths per minute. ------------------------------------------------------------------------------------------------------------------ \n average sleep time : 7.7Hours average duration in bed : 10.0Hours average heart rate : 62.6 average respiration rate : 13.4 average bed exit count : 1.1 Max bed Exits : 3 on 2023-03-20 Abnormality List :{'low Respiratory Rate': 'low Respiratory Rate of 4 on 2023-03-21 12:29:48', 'low Heart Rate': 'very low Heart Rate of 39 on 2023-03-16', 'low sleep duration': 'Sleep duration was lower than usual( 3 hours ) on 2023-03-19 10:22:49'} \n The resident slept an average of 7.7 hours a day over the past week which is within the recommended range of 7-12 hours. Additionally, they spent an average of 10 hours per day in their bed. On Saturday, March 18th the sleep duration was 3 hours which is lower than the recommended range. \n The resident exited the bed 1.1 times per night with the most bed exits occurring on Monday, March 20th with a total of 3. \n The heart rate throughout the week averaged at 62.6 beats per minute which is within the normal range of 50-100 beats per minute for an elderly adult. Thursday, March 16th the resident’s heart rate was at its lowest reaching 39 beats per minute. \n The respiratory rate throughout the week averaged at 13.4 breaths per minute, which is within the normal range of 8-30. On Tuesday, March 21st the resident’s respiratory rate was at its lowest reaching 4 breaths per minute. \n ----------------------------------------------------------------------------------------------------------------- \n average sleep time : 8.8Hours average duration in bed : 9.4Hours average heart rate : 63.2 average respiration rate : 14.7 average bed exit count : 0.0 Max bed Exits : 0 on Abnormality List :{'low Respiratory Rate': 'low Respiratory Rate of 6 on 2023-03-16 12:54:12', 'low Heart Rate': 'very low Heart Rate of 39 on 2023-03-19', 'low sleep duration': 'Sleep duration was lower than usual(4) on 2023-03-22 06:02:55', 'high heart Rate': 'high Heart Rate of 125 on 2023-03-22'} 22 06:02:55', 'high heart Rate': 'high Heart Rate of 125 on 2023-03-22'} \n The resident slept an average of 8.8 hours a day over the past week which is within the recommended range of 7-12 hours. Additionally, they spent an average of 9.4 hours per day in their bed. On Sunday, March 22nd the sleep duration was 4 hours which is lower than the recommended range. \n The resident slept well and did not exit the bed during their sleep this week. \n The heart rate throughout the week averaged at 63.2 beats per minute which is within the normal range of 50-100 beats per minute for an elderly adult. On Tuesday, March 19th the resident’s heart rate was at its lowest reaching 39 beats per minute. On Sunday, March 22nd the heart rate was at its highest reaching 125 beats per minute. \n The respiratory rate throughout the week averaged 14.7 breaths per minute, which is within the normal range of 8-30. On Monday, March 16th the resident’s respiratory rate was at its lowest reaching 6 breaths per minute."
res = openai.Completion.create(model=ft_model, prompt = prePrompt + postPrompt + '\n\n###\n\n', max_tokens=300, temperature=0)

In [280]:
print(res["choices"][0]["text"])



The resident slept an average of 6.02 hours a day over the past week which is within the recommended range of 7-12 hours. Additionally, they spent an average of 6.67 hours per day in their bed. On Wednesday, March 27th the sleep duration was 6 hours which is lower than the recommended range. The resident exited the bed 1 time per night with the most bed exits occurring on Tuesday, March 26th with a total of 1. The heart rate throughout the week averaged at 69 beats per minute which is within the normal range of 50-100 beats per minute for an elderly adult. On Wednesday, March 27th the resident’s heart rate was at its lowest reaching 39 beats per minute. The respiratory rate throughout the week averaged at 15 breaths per minute, which is within the normal range of 8-30. On Wednesday, March 27th the resident’s respiratory rate was at its lowest reaching 6 breaths per minute.

The sleep efficiency for this week was at 100% which is within the normal range of 85-110%. On Wednesday, March

In [276]:
print(postPrompt)

average sleep time : 6.02Hours
average duration in bed : 6.67Hours
average heart rate : 69
average respiration rate : 15
average bed exit count : 1
Max bed Exits  : 1 on 2023-03-26
Abnormality List :{'low respiratory rate': 'low Respiratory Rate of 6 on 2023-03-27 06:16:38', 'low sleep duration': 'Sleep duration was lower than usual (6.0 Hours ) on 2023-03-27 06:16:38'}
